In [6]:
#Here we import packages that we will need for this notebook. You can find out about these packages in the Help menu.

# although math is "built in" it needs to be imported so it's functions can be used.
import math

from scipy import constants, interpolate

#see numpy cheat sheet https://www.dataquest.io/blog/images/cheat-sheets/numpy-cheat-sheet.pdf
#The numpy import is needed because it is renamed here as np.
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

# sys and os give us access to operating system directory paths and to sys paths.
import sys, os

# If you place your GitHub directory in your documents folder and 
# clone both the design challenge notebook and the AguaClara_design repo, then this code should all work.
# If you have your GitHub directory at a different location on your computer, 
# then you will need to adjust the directory path below.
# add the path to your GitHub directory so that python can find files in other contained folders.
myGitHubdir=os.path.expanduser('~\\Documents\\Github')
if myGitHubdir not in sys.path:
    sys.path.append(myGitHubdir)

# add imports for AguaClara code that will be needed
# physchem has functions related to hydraulics, fractal flocs, flocculation, sedimentation, etc.
from AguaClara_design import physchem as pc

# pipedatabase has functions related to pipe diameters
from AguaClara_design import pipedatabase as pipe

# units allows us to include units in all of our calculations
from AguaClara_design.units import unit_registry as u

# utility has the significant digit display function
from AguaClara_design import utility as ut

ratio_VC_orifice= 0.62

The linear flow orifice meter was developed in 2008 by the AguaClara team at Cornell University in Ithaca, NY. It maintains a linear relationship between the height of water in the entrance tank and the overall plant flow rate, allowing an operator to easily determine how much water is flowing through the plant at any given time. This task is accomplished by mimicking the theoretical stout weir, as will be described below.
The LFOM made possible the development of an automatic chemical dose controller. This device utilizes the linear relationship between height and flow rate provided by the LFOM, and automatically doses the proper amount of coagulant to water leaving the entrance tank.
Below, we describe the algorithm which designs the AguaClara LFOM and then develop this algorithm using Mathcad code.

In [7]:
# The following constants need to go into the constants file
Pi_LFOM_safety = 1.2
# pipe schedule for LFOM
SDR_LFOM = 26

FLOW = 5*u.L/u.s
HL_LFOM = 20*u.cm

from enum import Enum

class uomeasure(Enum):
    english = 0
    metric = 1

def drill_series(uomeasure):
    if uomeasure is uomeasure.english:
        ds=np.arange(1/32, 1/4, 1/32)
        ds=np.append(ds,np.arange(3/8, 1, 1/8))
        ds=np.append(ds,np.arange(1.25, 3.25, 1/4))
        ds=ds*u.inch
    else:
        ds=np.arange(0.5, 4.9, 0.1)
        ds=np.append(ds,np.arange(5, 19, 1))
        ds=np.append(ds,np.arange(20, 50, 2))
        ds=ds*u.mm
    return ds

drill_series_uom = uomeasure

# Take the values of the array, compare to x, find the index of the first value less than or equal to x
def floor_nearest(x,array):
    myindex = np.argmax(array <= x)
    return array[myindex]

# Take the values of the array, compare to x, find the index of the first value greater or equal to x
def ceil_nearest(x,array):
    myindex = np.argmax(array >= x)
    return array[myindex]

In [8]:
class LFOM: 

	def __init__(self, flow, hl, ratio_safety, sdr, drill_series_uom):
		self.flow = flow
		self.hl = hl 
		self.ratio_safety = ratio_safety
		self.sdr = sdr
		self.drill_series_uom = drill_series_uom

	def set_flow(self, flow):
		self.flow = flow

	def set_hl(self, hl):
		self.hl = hl

	def set_ratio_safety(self, ratio_safety):
		self.ratio_safety = ratio_safety

	def set_sdr(self, sdr):
		self.sdr = sdr

	def set_drill_series(self, drill_series_uom):
		self.drill_series_uom = drill_series_uom


	def width_stout(self, z):
		return 2 / ((2 * u.g_0 * z)**(1/2) * math.pi * self.hl)

	def n_lfom_rows(self):
		n_est = (self.hl * math.pi / (2 * self.width_stout(self.hl) * self.flow)).to(u.dimensionless)
		return min(10, max(4, math.trunc(n_est.magnitude)))

	def dist_center_lfom_rows(self):
		return self.hl / self.n_lfom_rows()

	# average vertical velocity of the water inside the LFOM pipe 
    # at the very bottom of the bottom row of orifices
    # The speed of falling water is 0.841 m/s for all linear flow orifice meters of height 20cm,
    # independent of total plant flow rate.
    def vol_lfom_pipe_critical(self):
        return (4 / (3 * math.pi) * (2 * u.g_0 * self.hl)**(1/2)).to(u.m/u.s)

    def area_lfom_pipe_min(self):
        return (self.ratio_safety * self.flow / self.vol_lfom_pipe_critical()).to(u.m**2)

    def nom_diam_lfom_pipe(self):
        id = pc.diam_circle(self.area_lfom_pipe_min())
        return pipe.ND_SDR_available(id, self.sdr)

    # another possibility is to use integration to solve this problem.
    # Here we use the width of the stout weir in the center of the top row
    # to estimate the area of the top orifice
    def area_lfom_orifices_max(self):
        z = self.hl - 0.5 * self.dist_center_lfom_rows()
        return self.flow * self.width_stout(z) * self.dist_center_lfom_rows()

    def d_lfom_orifices_max(self):
        return pc.diam_circle(self.area_lfom_orifices_max())

    def lfom_drillbit_diameter(self):
        return ceil_nearest(self.d_lfom_orifices_max(), drill_series(self.drill_series_uom))

    def lfom_drillbit_area(self):
        return pc.area_circle(self.lfom_drillbit_diameter())

    ##A bound on the number of orifices allowed in each row.  
    ##The distance between consecutive orifices must be enough to retain structural integrity of the pipe
    def n_lfom_orifices_per_row_max(self):
        S_lfom_orifices_Min= 3 * u.mm
        nom_diam = self.nom_diam_lfom_pipe()
        drillbit_diam = self.lfom_drillbit_diameter() + S_lfom_orifices_Min
        return math.floor(math.pi * (pipe.ID_SDR(nom_diam, self.sdr)) / (drillbit_diam))

    #locations where we will try to get the target flows is in between orifices at elevation Pi.H
    def flow_ramp(self):
        dist_center = self.dist_center_lfom_rows() / u.cm
        return np.arange(dist_center, self.hl / u.cm, dist_center) * self.flow * u.cm / self.hl

    def height_lfom_orifices(self):
        drillbit_diam = self.lfom_drillbit_diameter() * 0.5
        return np.arange(drillbit_diam, self.hl, self.dist_center_lfom_rows(), dtype= object)

    #Calculate the flow for a given number of submerged rows of orifices
    def flow_lfom_actual(self, Row_Index_Submerged, N_LFOM_Orifices):
        D_LFOM_Orifices = self.lfom_drillbit_diameter().to(u.m)
        FLOW_new=[]
        dist_center = self.dist_center_lfom_rows()
        for i in range(Row_Index_Submerged):
            h = np.arange(dist_center, self.hl, dist_center, dtype=object)
            h = h[Row_Index_Submerged].to(u.m)
            d = np.arange(0.5* D_LFOM_Orifices, self.hl, dist_center, dtype=object)
            FLOW_new.append(N_LFOM_Orifices[i]*(pc.flow_orifice_vert(D_LFOM_Orifices, h - d[i], ratio_VC_orifice)))
        return sum(FLOW_new)

    #Calculate number of orifices at each level given a diameter
    def fric_n_lfom_orifices(self):
        FLOW_ramp_local = self.flow_ramp()
        D_LFOM_Orifices = self.lfom_drillbit_diameter()
        H_ramp_local = np.arange(D_LFOM_Orifices * 0.5, self.hl, self.dist_center_lfom_rows(), dtype=object)
        n = []
        for i in range (len(H_ramp_local) - 1):
            h = np.arange(self.dist_center_lfom_rows(), self.hl, self.dist_center_lfom_rows(), dtype=object)
            d = H_ramp_local
            flow_actual = self.flow_lfom_actual(i, n)
            if self.nom_diam_lfom_pipe() <= 12 * u.inch:
                rounded = round((FLOW_ramp_local[i] - flow_actual) / pc.flow_orifice_vert(D_LFOM_Orifices, h[i] - d[i], ratio_VC_orifice))
                n.append(min(max(0, rounded), self.n_lfom_orifices_per_row_max()))
            else:
                 n.append(max(0,rounded))
        return n

    #This function calculates the error of the design based on the differences between the predicted flow rate
    #and the actual flow rate through the LFOM.
    def flow_lfom_error(self):
        N_lfom_orifices = self.fric_n_lfom_orifices()
        FLOW_lfom_error = []
        for j in range (len(N_lfom_orifices) - 1):
            flow_actual = self.flow_lfom_actual(j, N_lfom_orifices)
            FLOW_lfom_error.append((flow_actual - self.flow_ramp()[j]) / self.flow)
        return FLOW_lfom_error

    def flow_lfom_error_max(self):
        x = max(self.flow_lfom_error())
        y = x**2
        return y**1/2

    def flow_lfom_ideal(self, h):
        flow_lfom_ideal=(self.flow * h) / self.hl
        return flow_lfom_ideal
    
    def flow_lfom(self, h):
        D_lfom_orifices = self.lfom_drillbit_diameter()
        H_submerged = np.arange(h - 0.5 * D_lfom_orifices, self.hl, h - self.dist_center_lfom_rows(), dtype=object)
        N_lfom_orifices = self.fric_n_lfom_orifices()
        flow = []
        for i in range (len(H_submerged)):
            flow.append(pc.flow_orifice_vert(D_lfom_orifices, H_submerged[i], ratio_VC_orifice) * N_lfom_orifices[i])
        return sum(flow)

In [9]:
print(drill_series(drill_series_uom))
print(drill_series(drill_series_uom.english))
print(drill_series(drill_series_uom.metric))


[  0.5   0.6   0.7   0.8   0.9   1.    1.1   1.2   1.3   1.4   1.5   1.6   1.7   1.8   1.9   2.    2.1   2.2   2.3   2.4   2.5   2.6   2.7   2.8   2.9   3.    3.1   3.2   3.3   3.4   3.5   3.6   3.7   3.8   3.9   4.   4.1   4.2   4.3   4.4   4.5   4.6   4.7   4.8   5.    6.    7.    8.   9.   10.   11.   12.   13.   14.   15.   16.   17.   18.   20.   22.  24.   26.   28.   30.   32.   34.   36.   38.   40.   42.   44.   46.  48. ] millimeter
[ 0.03125  0.0625   0.09375  0.125    0.15625  0.1875   0.21875  0.375    0.5  0.625    0.75     0.875    1.25     1.5      1.75     2.       2.25     2.5  2.75     3.     ] inch
[  0.5   0.6   0.7   0.8   0.9   1.    1.1   1.2   1.3   1.4   1.5   1.6   1.7   1.8   1.9   2.    2.1   2.2   2.3   2.4   2.5   2.6   2.7   2.8   2.9   3.    3.1   3.2   3.3   3.4   3.5   3.6   3.7   3.8   3.9   4.   4.1   4.2   4.3   4.4   4.5   4.6   4.7   4.8   5.    6.    7.    8.   9.   10.   11.   12.   13.   14.   15.   16.   17.   18.   20.   22.  24.   26.   28.

In [10]:
# testing to make sure everything still is the same

lfom = LFOM()

lfom.n_lfom_rows(FLOW,HL_LFOM)

10

In [28]:
(lfom.width_stout(HL_LFOM,HL_LFOM)*FLOW).to(u.m)/u.m

<Quantity(0.008035812563075291, 'dimensionless')>

In [29]:
lfom.dist_center_lfom_rows(FLOW,HL_LFOM)

<Quantity(2.0, 'centimeter')>

In [30]:
lfom.vol_lfom_pipe_critical(HL_LFOM)

<Quantity(0.8405802802312781, 'meter / second')>

In [31]:
lfom.area_lfom_pipe_min(FLOW,HL_LFOM,Pi_LFOM_safety)

<Quantity(0.007137926193497133, 'meter ** 2')>

In [32]:
lfom.nom_diam_lfom_pipe(11*u.L/u.s,HL_LFOM,Pi_LFOM_safety,SDR_LFOM)

<Quantity(6.0, 'inch')>

In [33]:
(FLOW*lfom.width_stout(HL_LFOM,HL_LFOM-0.5*lfom.dist_center_lfom_rows(FLOW,HL_LFOM))).to(u.mm)

<Quantity(8.244569731129168, 'millimeter')>

In [34]:
(lfom.d_lfom_orifices_max(FLOW,HL_LFOM)).to(u.mm)

<Quantity(14.489521876859984, 'millimeter')>

In [14]:
print(lfom.lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom.english))
print(lfom.lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom.metric))
print(lfom.lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom))

0.625 inch
15.0 millimeter
15.0 millimeter


In [15]:
print(lfom.lfom_drillbit_area(FLOW,HL_LFOM,drill_series_uom.english))
print(lfom.lfom_drillbit_area(FLOW,HL_LFOM,drill_series_uom.metric))
print(lfom.lfom_drillbit_area(FLOW,HL_LFOM,drill_series_uom))

0.30679615757712825 inch ** 2
176.71458676442586 millimeter ** 2
176.71458676442586 millimeter ** 2


In [37]:
lfom.n_lfom_orifices_per_row_max(FLOW,HL_LFOM,drill_series_uom.metric,SDR_LFOM)

18

In [38]:
lfom.flow_ramp(FLOW,HL_LFOM)

<Quantity([ 0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5], 'liter / second')>

In [39]:
lfom.height_lfom_orifices(FLOW,HL_LFOM,drill_series_uom.metric) 

array([<Quantity(7.5, 'millimeter')>, <Quantity(27.5, 'millimeter')>,
       <Quantity(47.5, 'millimeter')>, <Quantity(67.5, 'millimeter')>,
       <Quantity(87.5, 'millimeter')>, <Quantity(107.5, 'millimeter')>,
       <Quantity(127.5, 'millimeter')>, <Quantity(147.5, 'millimeter')>,
       <Quantity(167.5, 'millimeter')>, <Quantity(187.5, 'millimeter')>], dtype=object)

In [40]:
lfom.fric_n_lfom_orifices(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM)

[<Quantity(9.0, 'dimensionless')>,
 <Quantity(4.0, 'dimensionless')>,
 <Quantity(3.0, 'dimensionless')>,
 <Quantity(2.0, 'dimensionless')>,
 <Quantity(3.0, 'dimensionless')>,
 <Quantity(1.0, 'dimensionless')>,
 <Quantity(3.0, 'dimensionless')>,
 <Quantity(1.0, 'dimensionless')>,
 <Quantity(1.0, 'dimensionless')>]

In [41]:
lfom.flow_lfom_error(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM)

[<Quantity(-0.1, 'dimensionless')>,
 <Quantity(-0.04281022675343118, 'dimensionless')>,
 <Quantity(-0.03014536546205875, 'dimensionless')>,
 <Quantity(-0.023627100599359574, 'dimensionless')>,
 <Quantity(-0.028341082805428908, 'dimensionless')>,
 <Quantity(-0.013856233840633347, 'dimensionless')>,
 <Quantity(-0.027031604731054416, 'dimensionless')>,
 <Quantity(-0.008034482759309292, 'dimensionless')>]

In [42]:
lfom.flow_lfom_error_max(FLOW, HL_LFOM, drill_series_uom, SDR_LFOM)

<Quantity(3.227645660481912e-05, 'dimensionless')>

In [43]:
lfom.flow_lfom_ideal(FLOW,HL_LFOM,3)

<Quantity(0.75, 'liter / centimeter / second')>

In [44]:
lfom.flow_lfom(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM,5*u.cm)

<Quantity(3.04063042951175, 'liter / second')>

In [46]:
HEIGHT_LFOM_ORIFICES=lfom.height_lfom_orifices(FLOW,HL_LFOM,drill_series_uom)
if FLOW==1.6*(u.L/u.s):
   NOM_DIAM_RAPID_MIX_pipe=2*u.inch
else:
   NOM_DIAM_RAPID_MIX_pipe=lfom.nom_diam_lfom_pipe(11*u.L/u.s,HL_LFOM,Pi_LFOM_safety,SDR_LFOM)
N_LFOM_ORIFICES=lfom.fric_n_lfom_orifices(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM)
N_LFOM_ROWS=len(N_LFOM_ORIFICES)

print(HEIGHT_LFOM_ORIFICES)
print(N_LFOM_ORIFICES)
print(N_LFOM_ROWS)

[<Quantity(7.5, 'millimeter')> <Quantity(27.5, 'millimeter')>
 <Quantity(47.5, 'millimeter')> <Quantity(67.5, 'millimeter')>
 <Quantity(87.5, 'millimeter')> <Quantity(107.5, 'millimeter')>
 <Quantity(127.5, 'millimeter')> <Quantity(147.5, 'millimeter')>
 <Quantity(167.5, 'millimeter')> <Quantity(187.5, 'millimeter')>]
[<Quantity(9.0, 'dimensionless')>, <Quantity(4.0, 'dimensionless')>, <Quantity(3.0, 'dimensionless')>, <Quantity(2.0, 'dimensionless')>, <Quantity(3.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(3.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>]
9
